In [34]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob

In [35]:
import tkinter as tk
from tkinter import filedialog, messagebox

## Select folders

In [36]:
root = tk.Tk()
root.withdraw()
messagebox.showinfo("Folder Selection", "Press OK and then select the master folder (raw images)")

photos_dir_master = filedialog.askdirectory(title="Select the master folder")
photos_dir_master

'/Users/semenzalab/Desktop/master folder'

In [37]:
if not photos_dir_master:
    raise ValueError("No input provided")

In [38]:
# folder containing csv files with results from ImageJ
messagebox.showinfo("Folder Selection", "Press OK and then select the results folder (containing all csv files to be analyzed)")
results_dir = filedialog.askdirectory(title="Select the folder containing resulting csv files from ImageJ")
results_dir

'/Users/semenzalab/Desktop/master folder__bw_output/Results'

In [39]:
if not results_dir:
    raise ValueError("No input provided")

# Run cells below

In [40]:
# folder path you want to store processed results in
output_dir = os.path.join(os.path.dirname(results_dir), "Processed Results_cellarea")
os.makedirs(output_dir)

In [41]:
images_per_figure = []
figures = [folder for folder in os.listdir(photos_dir_master) if os.path.isdir(os.path.join(photos_dir_master, folder))]

In [42]:
results = glob.glob(os.path.join(results_dir, "*.csv"))

In [43]:
cellarea_dct_sq = {}
for i in range(len(figures)):
    csv_filename = os.path.join(results_dir, figures[i] + ".csv")
    cellnums = pd.read_csv(csv_filename, index_col=0)
    image_names_i = glob.glob(os.path.join(photos_dir_master + "/" + figures[i], "**", "*.tif"), recursive=True)
    image_names_i = [j.split("/")[-1] for j in image_names_i]
    cellarea_dct = {j: sum(cellnums.loc[cellnums["Label"] == f"outliers_{j}", "Area"]) for j in image_names_i}
    cellarea_dct_sq.update({figures[i]: cellarea_dct})

In [44]:
cellarea_dct_sq

{'9. ITGB3-Fig.S3D-Invasion': {'Fig.S3D-Invasion-EV-Re2-1.tif': 862377,
  'Fig.S3D-Invasion-EV-Re2-2.tif': 984513,
  'Fig.S3D-Invasion-EV-Re2-3.tif': 920025,
  'Fig.S3D-Invasion-EV-Re3-2.tif': 186850,
  'Fig.S3D-Invasion-EV-Re3-3.tif': 145194,
  'Fig.S3D-Invasion-EV-Re1-1.tif': 1220522,
  'Fig.S3D-Invasion-EV-Re1-3.tif': 933026,
  'Fig.S3D-Invasion-EV-Re3-1.tif': 316659,
  'Fig.S3D-Invasion-EV-Re1-2.tif': 1003310,
  'Fig.S3D-Invasion-OE-Re1-1.tif': 1690818,
  'Fig.S3D-Invasion-OE-Re3-3.tif': 2481366,
  'Fig.S3D-Invasion-OE-Re3-2.tif': 2374015,
  'Fig.S3D-Invasion-OE-Re1-2.tif': 1581900,
  'Fig.S3D-Invasion-OE-Re3-1.tif': 2212536,
  'Fig.S3D-Invasion-OE-Re1-3.tif': 1778209,
  'Fig.S3D-Invasion-OE-Re2-1.tif': 2528954,
  'Fig.S3D-Invasion-OE-Re2-3.tif': 2396563,
  'Fig.S3D-Invasion-OE-Re2-2.tif': 2243988},
 '2. ITGB3-Fig.3G-Invasion': {'Fig.3G-Invasion-shHIF1A-20%-Re1-3.tif': 383062,
  'Fig.3G-Invasion-shHIF1A-20%-Re3-1.tif': 353831,
  'Fig.3G-Invasion-shHIF1A-20%-Re1-2.tif': 385644,
  'F

In [45]:
names = [f"{i}-{j}" for i in range(1,4) for j in range(1,4)]
names_br = list(range(1,4))
#image_size = 4080 * 3072

for folder in cellarea_dct_sq.keys():
    cellarea_df = pd.DataFrame.from_dict(cellarea_dct_sq[folder], orient="index").reset_index()
    cellarea_df.columns = ["image_name", "cell_area"]
    cellarea_df["cell_area"] = cellarea_df["cell_area"] #/ image_size
    
    
    df = cellarea_df.copy()
    df.set_index("image_name", inplace=True)
    reform_idx = [j.lower() for j in df.index]
    df.index = reform_idx
    print(df)
    bio_groups = np.unique([j[:-7] for j in reform_idx])
    new_idx = [f"{bio_group}{name}.tif" for bio_group in bio_groups for name in names]
    new_dct = {new_id: df.loc[new_id, "cell_area"] for new_id in new_idx}
    new_vals = np.array(list(new_dct.values())).reshape(-1,3)
    new_df = pd.DataFrame(new_vals, columns=["TR1", "TR2", "TR3"], 
                          index=[f"{bio_group}{k}" for bio_group in bio_groups for k in names_br]).\
                            reset_index()
    new_df["average"] = new_df.loc[:, ["TR1", "TR2", "TR3"]].mean(axis=1)

    with pd.ExcelWriter(f"{output_dir}/{folder}_processed.xlsx") as f:
        cellarea_df.to_excel(f, sheet_name="cell_area")
        new_df.to_excel(f, sheet_name="re-formatted")

        

                               cell_area
fig.s3d-invasion-ev-re2-1.tif     862377
fig.s3d-invasion-ev-re2-2.tif     984513
fig.s3d-invasion-ev-re2-3.tif     920025
fig.s3d-invasion-ev-re3-2.tif     186850
fig.s3d-invasion-ev-re3-3.tif     145194
fig.s3d-invasion-ev-re1-1.tif    1220522
fig.s3d-invasion-ev-re1-3.tif     933026
fig.s3d-invasion-ev-re3-1.tif     316659
fig.s3d-invasion-ev-re1-2.tif    1003310
fig.s3d-invasion-oe-re1-1.tif    1690818
fig.s3d-invasion-oe-re3-3.tif    2481366
fig.s3d-invasion-oe-re3-2.tif    2374015
fig.s3d-invasion-oe-re1-2.tif    1581900
fig.s3d-invasion-oe-re3-1.tif    2212536
fig.s3d-invasion-oe-re1-3.tif    1778209
fig.s3d-invasion-oe-re2-1.tif    2528954
fig.s3d-invasion-oe-re2-3.tif    2396563
fig.s3d-invasion-oe-re2-2.tif    2243988
                                          cell_area
fig.3g-invasion-shhif1a-20%-re1-3.tif        383062
fig.3g-invasion-shhif1a-20%-re3-1.tif        353831
fig.3g-invasion-shhif1a-20%-re1-2.tif        385644
fig.3g-invasi

In [46]:
new_df

,index,TR1,TR2,TR3,average
0,fig.s4b-reo-migration-1%-re1,2557451,2882266,3130395,2.856704e+06
1,fig.s4b-reo-migration-1%-re2,4134179,3509592,3929174,3.857648e+06
2,fig.s4b-reo-migration-1%-re3,3476157,2575544,3164852,3.072184e+06
3,fig.s4b-reo-migration-20%-re1,1702383,1827368,1326805,1.618852e+06
4,fig.s4b-reo-migration-20%-re2,835096,979234,1001254,9.385280e+05
5,fig.s4b-reo-migration-20%-re3,1667781,1694226,1776074,1.712694e+06
6,fig.s4d-reo-invasion-1%-re1,1786659,1723954,1814650,1.775088e+06
7,fig.s4d-reo-invasion-1%-re2,1201863,1387248,1309983,1.299698e+06
8,fig.s4d-reo-invasion-1%-re3,1486058,1485873,1795581,1.589171e+06
9,fig.s4d-reo-invasion-20%-re1,1009582,653058,403742,6.887940e+05
